# Gathering all Candidate Data

This notebook collects all of the candidate data from 2018 until 2023 at the CM and PG level. 

In [1]:
import pandas as pd
import requests
from ingester3.extensions import *
from ingester3.DBWriter import DBWriter
from ingester3.scratch import cache_manager
from ingester3.config import source_db_path
from diskcache import Cache


# This cell imports the basic packages needed to run the notebook 

import numpy as np
import pandas as pdx
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import tabula
import xlwings as xw
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
#import views_dataviz
from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *

# VIEWS mapper2
#from views_mapper2.label_writer import *

# Ingester
from ingester3.config import source_db_path
from ingester3.Country import Country
from ingester3.extensions import *
from ingester3.ViewsMonth import ViewsMonth


#from tabula import read_pdf
#from tabulate import tabulate
#import plotly.express as px


import os
home = os.path.expanduser("~")


##################
print('OS Login is:', os.getlogin())
print('OS path is set to:', home)

OS Login is: root
OS path is set to: /Users/chandlerwilliams


In [ ]:
print(source_db_path)

In [ ]:
ged_cache = Cache('ged_x.cache')
ged_cache.clear(retry=True)


class GedLoader:
    def __init__(self,version,verbose=True, no_val=None):
        
        cache_manager()
        self.version = version
        self.ged = None
        
        self.ged_agg_pgm = None
        self.ged_agg_cm = None

        if no_val is None: 
            self.no_val = []
        else:
            self.no_val = [i.lower() for i in no_val]
            
        self.__get_month_id()
        
        self.verbose_print = print if verbose else lambda *a, **k: None

        
    def __get_month_id(self):
        """
        If trying to load a GED Candidates dataset (20.0.x) infer what ViEWS MonthID it refers to.
        Return nothing otherwise
        """
        self.month_id = None
        if self.version.count('.')==2:
            year_extent=int('20'+self.version.split('.')[0])
            month_extent=int(self.version.split('.')[2])
            self.month_id = ViewsMonth.from_year_month(year_extent,month_extent)
    
    @staticmethod
    @ged_cache.memoize(typed=True, expire=600000, tag='ged_slice') 
    def _get_ged_slice(next_page_url, token=None):
        headers = {'x-ucdp-access-token': token}
        r = requests.get(next_page_url, headers=headers)
        output = r.json()
        next_page_url = output['NextPageUrl'] if output['NextPageUrl'] != '' else None
        ged = pd.DataFrame(output['Result'])
        page_count = output['TotalPages']
        return next_page_url, ged, page_count
    
    def fetch_ged(self, pagesize=50000):
        cur_page = 1
        next_page_url = f"https://ucdpapi.pcr.uu.se/api/gedevents/{self.version}?pagesize={pagesize}&page=0"

        df = pd.DataFrame()
        while next_page_url:
            print(next_page_url)
            next_page_url, ged_slice, total_pages = self._get_ged_slice(\
                next_page_url=next_page_url, token="48dda3460c347f3b"\
            )
            df = pd.concat([df,ged_slice], ignore_index=True)
            print(f"{cur_page} of {total_pages} pages loaded.")
            #cur_page += 1
            
            if cur_page > total_pages:
                ged_cache.clear(retry=True)
                raise ConnectionError('The UCDP API is misbehaving. Try again later!')
            cur_page += 1
            

        self.ged = df
        
    def filter_ged(self):
        self.ged = self.ged[self.ged.priogrid_gid>=1]
        self.ged.date_end = pd.to_datetime(self.ged.date_end)
        self.ged = pd.DataFrame.pgm.from_datetime(self.ged,'date_end').rename(columns = {'priogrid_gid':'pg_id',
                                                                       'type_of_violence':'tv'})
        self.ged = self.ged[self.ged.tv<4]
        self.pg_ged = self.ged[(self.ged.where_prec != 4) & (self.ged.where_prec != 6)]
        
    def aggregate_to_pg(self):
        """
        Aggregate GED to Priogrid level.
        """
        
        #Group by type of violence, priogrid id and month id
        #Aggregate to PG, taking sum and count
        ged_fraction = self.pg_ged[['tv','pg_id','month_id','best','high']]
        ged_agg = ged_fraction.groupby(by=['tv','pg_id','month_id']).aggregate(['sum','count'])
        
        #Eliminate the double-index nesting that resulted from the aggregation process
        ged_agg['best_sum'] = ged_agg['best']['sum']
        ged_agg['best_count'] = ged_agg['best']['count']
        ged_agg['high_sum'] = ged_agg['high']['sum']
        ged_agg['high_count'] = ged_agg['high']['count']
        del ged_agg['best']
        del ged_agg['high']
        
        #Reset the index
        ged_agg = ged_agg.reset_index()
        
        #Pivot the long-form to a wide-form required by the database
        #You don't need to fill in the panel because the DB will automate the infill.
        ged_pivot = ged_agg.pivot(index=['pg_id','month_id'], 
              columns=['tv'],
              values = ['best_sum','best_count','high_sum','high_count']).fillna(0).astype('int64')
        
        #The pivot will produce a multi-level columnar structure
        #Flatten this to column names that we will be using in the DB.
        
        ged_pivot['ged_sb_best_sum_nokgi'] = ged_pivot['best_sum'][1]
        ged_pivot['ged_ns_best_sum_nokgi'] = ged_pivot['best_sum'][2]
        ged_pivot['ged_os_best_sum_nokgi'] = ged_pivot['best_sum'][3]

        ged_pivot['ged_sb_best_count_nokgi'] = ged_pivot['best_count'][1]
        ged_pivot['ged_ns_best_count_nokgi'] = ged_pivot['best_count'][2]
        ged_pivot['ged_os_best_count_nokgi'] = ged_pivot['best_count'][3]

        ged_pivot['ged_sb_high_sum_nokgi'] = ged_pivot['high_sum'][1]
        ged_pivot['ged_ns_high_sum_nokgi'] = ged_pivot['high_sum'][2]
        ged_pivot['ged_os_high_sum_nokgi'] = ged_pivot['high_sum'][3]

        ged_pivot['ged_sb_high_count_nokgi'] = ged_pivot['high_count'][1]
        ged_pivot['ged_ns_high_count_nokgi'] = ged_pivot['high_count'][2]
        ged_pivot['ged_os_high_count_nokgi'] = ged_pivot['high_count'][3]

        del(ged_pivot['best_count'])
        del(ged_pivot['high_count'])
        del(ged_pivot['best_sum'])
        del(ged_pivot['high_sum'])
        
        #Simplify everything by removing the multi-level columnar structure.
        ged_pivot = ged_pivot.reset_index()
        ged_pivot.columns = ged_pivot.columns.droplevel(1)
        
        self.ged_agg_pgm = ged_pivot
        return self.ged_agg_pgm
    
    
    def aggregate_to_cm(self):
        
        ged_cm_agg = self.ged[['tv','country_id','month_id','best','high']].\
        groupby(by=['tv','country_id','month_id']).aggregate(['sum','count'])

        ged_cm_agg['best_sum'] = ged_cm_agg['best']['sum']
        ged_cm_agg['best_count'] = ged_cm_agg['best']['count']
        ged_cm_agg['high_sum'] = ged_cm_agg['high']['sum']
        ged_cm_agg['high_count'] = ged_cm_agg['high']['count']

        del ged_cm_agg['best']
        del ged_cm_agg['high']

        ged_cm_agg = ged_cm_agg.reset_index()
        
        # GED is sometimes faulty in terms of what countries and months it contains.
        # We need to filter out the not working data.
        
        ged_cm_agg = pd.DataFrame.cm.soft_validate_gwcode(ged_cm_agg,'country_id','month_id')
        ged_cm_agg = ged_cm_agg[ged_cm_agg.valid_id==True]
        
        ged_cm_agg = pd.DataFrame.cm.from_gwcode(ged_cm_agg, 
                                                 gw_col='country_id', 
                                                 month_col='month_id')

        ged_cm_agg.columns = ged_cm_agg.columns.droplevel(1)
        del ged_cm_agg['country_id']
        
        ged_cm_pivot = ged_cm_agg.pivot(index=['c_id','month_id'], 
                                columns=['tv'],
                                values = ['best_sum','best_count',
                                          'high_sum','high_count']).fillna(0).astype('int64')
        
        ged_cm_pivot['ged_sb_best_sum_nokgi'] = ged_cm_pivot['best_sum'][1]
        ged_cm_pivot['ged_ns_best_sum_nokgi'] = ged_cm_pivot['best_sum'][2]
        ged_cm_pivot['ged_os_best_sum_nokgi'] = ged_cm_pivot['best_sum'][3]

        ged_cm_pivot['ged_sb_best_count_nokgi'] = ged_cm_pivot['best_count'][1]
        ged_cm_pivot['ged_ns_best_count_nokgi'] = ged_cm_pivot['best_count'][2]
        ged_cm_pivot['ged_os_best_count_nokgi'] = ged_cm_pivot['best_count'][3]
        
        ged_cm_pivot['ged_sb_high_sum_nokgi'] = ged_cm_pivot['high_sum'][1]
        ged_cm_pivot['ged_ns_high_sum_nokgi'] = ged_cm_pivot['high_sum'][2]
        ged_cm_pivot['ged_os_high_sum_nokgi'] = ged_cm_pivot['high_sum'][3]

        ged_cm_pivot['ged_sb_high_count_nokgi'] = ged_cm_pivot['high_count'][1]
        ged_cm_pivot['ged_ns_high_count_nokgi'] = ged_cm_pivot['high_count'][2]
        ged_cm_pivot['ged_os_high_count_nokgi'] = ged_cm_pivot['high_count'][3]

        del(ged_cm_pivot['best_count'])
        del(ged_cm_pivot['high_count'])
        del(ged_cm_pivot['best_sum'])
        del(ged_cm_pivot['high_sum'])

        ged_cm_pivot = ged_cm_pivot.reset_index()
        ged_cm_pivot.columns = ged_cm_pivot.columns.droplevel(1)
        
        self.ged_agg_cm = ged_cm_pivot
        return self.ged_agg_cm


In [ ]:
GED_VERSIONS = [
    "18.0.1", "18.0.2", "18.0.3", "18.0.4", "18.0.5", "18.0.6",
    "18.0.7", "18.0.8", "18.0.9", "18.0.10", "18.0.11", "18.0.12",
    "19.0.1", "19.0.2", "19.0.3", "19.0.4", "19.0.5", "19.0.6",
    "19.0.7", "19.0.8", "19.0.9", "19.0.10", "19.0.11", "19.0.12",
    "20.0.1", "20.0.2", "20.0.3", "20.0.4", "20.0.5", "20.0.6",
    "20.0.7", "20.0.8", "20.0.9", "20.0.10", "20.0.11", "20.0.12",
    "21.0.1", "21.0.2", "21.0.3", "21.0.4", "21.0.5", "21.0.6",
    "21.0.7", "21.0.8", "21.0.9", "21.0.10", "21.0.11", "21.0.12",
    "22.0.1", "22.0.2", "22.0.3", "22.0.4", "22.0.5", "22.0.6",
    "22.0.7", "22.0.8", "22.0.9", "22.0.10", "22.0.11", "22.0.12",
    "23.0.1", "23.0.2", "23.0.3", "23.0.4", "23.0.5", "23.0.6",
    "23.0.7", "23.0.8", "23.0.9", "23.0.10", "23.0.11", "23.0.12"
]

In [ ]:
# Loop through each GED version
for version in GED_VERSIONS:
    print(f"\n🚀 Processing GED Version: {version}")

    # Initialize the loader
    loader = GedLoader(version)

    # Fetch and filter data
    loader.fetch_ged()
    loader.filter_ged()

    # Aggregate data
    pg_df = loader.aggregate_to_pg()
    cm_df = loader.aggregate_to_cm()

    # 🔹 Find the most common `month_id` for both datasets
    most_common_month_pg = pg_df["month_id"].mode()[0]  # Priogrid
    most_common_month_cm = cm_df["month_id"].mode()[0]  # Country-Month

    # 🔹 Filter to keep only rows with the most common `month_id`
    pg_df_filtered = pg_df[pg_df["month_id"] == most_common_month_pg]
    cm_df_filtered = cm_df[cm_df["month_id"] == most_common_month_cm]

    # Save filtered datasets
    pg_df_filtered.to_csv(f"candidate_{version.replace('.', '_')}_pgm_filtered.csv", index=False)
    cm_df_filtered.to_csv(f"candidate_{version.replace('.', '_')}_cm_filtered.csv", index=False)

    print(f"✅ Saved: candidate_{version.replace('.', '_')}_pgm_filtered.csv ({len(pg_df_filtered)} rows)")
    print(f"✅ Saved: candidate_{version.replace('.', '_')}_cm_filtered.csv ({len(cm_df_filtered)} rows)")

# Combining these all into a single df

In [ ]:
import pandas as pd
import glob

# Find all CM filtered CSV files
cm_files = glob.glob("candidate_*_cm_filtered.csv")

# List to store dataframes
df_list = []

# Load each CM file and append it to the list
for file in cm_files:
    print(f"📂 Loading: {file}")
    df = pd.read_csv(file)
    df["source_version"] = file  # Add a column to track the source file
    df_list.append(df)

# Concatenate all DataFrames
combined_df = pd.concat(df_list, ignore_index=True)

# Save the combined DataFrame
combined_df.to_csv("candidate_combined_cm_filtered.csv", index=False)

print(f"✅ Combined DataFrame saved: candidate_combined_cm_filtered.csv ({len(combined_df)} rows)")


## From here down, getting the nowcasting data together

In [2]:
# load combined_df.to_csv("candidate_combined_cm_filtered.csv") and index by , index= month_id and c_id
combined_df = pd.read_csv("candidate_combined_cm_filtered.csv")

# index df by month_id and c_id
combined_df = combined_df.set_index(['month_id', 'c_id'])


In [7]:
# shwo the first 5 rows
combined_df.head(10)


ged_sb_best_sum_nokgi  ged_ns_best_sum_nokgi  \
month_id c_id                                                 
492      9                         0                     22   
         13                        0                      0   
         17                        0                    245   
         24                        0                      0   
         41                        1                      0   
         47                       19                     10   
         48                        0                     17   
         49                        0                      4   
         50                       23                     44   
         57                      210                    258   

               ged_os_best_sum_nokgi  ged_sb_best_count_nokgi  \
month_id c_id                                                   
492      9                         0                        0   
         13                        3                        0   
         17                        1                        0   
         24                        3                        0   
         41                        0                        1   
         47                        5                        4   
         48                        0                        0   
         49                        0                        0   
         50                        2                        8   
         57                       36                        2   

               ged_ns_best_count_nokgi  ged_os_best_count_nokgi  \
month_id c_id                                                     
492      9                          18                        0   
         13                          0                        2   
         17                        149                        1   
         24                          0                        2   
         41                          0                        0   
         47                          1                        3   
         48                          1                        0   
         49                          1                        0   
         50                          7                        1   
         57                          2                        5   

               ged_sb_high_sum_nokgi  ged_ns_high_sum_nokgi  \
month_id c_id                                                 
492      9                         0                     22   
         13                        0                      0   
         17                        0                    245   
         24                        0                      0   
         41                        1                      0   
         47                       19                     10   
         48                        0                     17   
         49                        0                      4   
         50                       25                     44   
         57                      210                    258   

               ged_os_high_sum_nokgi  ged_sb_high_count_nokgi  \
month_id c_id                                                   
492      9                         0                        0   
         13                        3                        0   
         17                        1                        0   
         24                        3                        0   
         41                        0                        1   
         47                        5                        4   
         48                        0                        0   
         49                        0                        0   
         50                        2                        8   
         57                       36                        2   

               ged_ns_high_count_nokgi  ged_os_high_count_nokgi  \
month_id c_id                                                     


# Gather all UCDP Data from VIEWSER (GED FINAL)

## Task 2: Read in UCDP-GED for 1990—2022 
1. GED best and GED count 
2. All three types: sb, os, ns

In [8]:
# 2. UCDP GED Final, ACLED, Topics

qs_nowcasting_cm = (Queryset("nowcasting", "country_month") 
                    
# COUNTRY META-DATA       
.with_column(Column('country_name', from_loa = 'country', from_column = 'name'))
.with_column(Column('iso_ab', from_loa = 'country', from_column = 'isoab'))
.with_column(Column('gleditsch_ward', from_loa = 'country', from_column = 'gwcode'))
.with_column(Column('year', from_loa = 'month', from_column = 'year_id'))
.with_column(Column('month', from_loa = 'month', from_column = 'month'))

# UCDP GED FINAL                    
.with_column(Column('sb_final_best', from_loa = 'country_month', from_column = 'ged_sb_best_sum_nokgi'))
.with_column(Column('ns_final_best', from_loa = 'country_month', from_column = 'ged_ns_best_sum_nokgi'))
.with_column(Column('os_final_best', from_loa = 'country_month', from_column = 'ged_os_best_sum_nokgi'))
.with_column(Column('sb_ged_count_events', from_loa = 'country_month', from_column = 'ged_sb_best_count_nokgi'))
.with_column(Column('ns_ged_count_events', from_loa = 'country_month', from_column = 'ged_ns_best_count_nokgi'))
.with_column(Column('os_ged_count_events', from_loa = 'country_month', from_column = 'ged_os_best_count_nokgi'))  
                    
# Logged VERSIONS
.with_column(Column("sb_final_best_ln", from_loa = "country_month", from_column = "ged_sb_best_sum_nokgi")
                 .transform.ops.ln()
                 .transform.missing.replace_na()) 
                    
.with_column(Column("ns_final_best_ln", from_loa = "country_month", from_column = "ged_ns_best_sum_nokgi")
                 .transform.ops.ln()
                 .transform.missing.replace_na()) 
                    
.with_column(Column("os_final_best_ln", from_loa = "country_month", from_column = "ged_os_best_sum_nokgi")
                 .transform.ops.ln()
                 .transform.missing.replace_na()) 

# LAGGED VERSIONS
.with_column(Column("sb_final_best_ln_1", from_loa = "country_month", from_column = "ged_sb_best_sum_nokgi")
                 .transform.ops.ln()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(1)
                 .transform.missing.replace_na()) 

.with_column(Column("sb_final_best_ln_2", from_loa = "country_month", from_column = "ged_sb_best_sum_nokgi")
                 .transform.ops.ln()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(2)
                 .transform.missing.replace_na())
                    
.with_column(Column("sb_final_best_ln_3", from_loa = "country_month", from_column = "ged_sb_best_sum_nokgi")
                 .transform.ops.ln()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(3)
                 .transform.missing.replace_na())
                    
.with_column(Column("sb_final_best_ln_4", from_loa = "country_month", from_column = "ged_sb_best_sum_nokgi")
                 .transform.ops.ln()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(4)
                 .transform.missing.replace_na())
                    
.with_column(Column("sb_final_best_ln_6", from_loa = "country_month", from_column = "ged_sb_best_sum_nokgi")
                 .transform.ops.ln()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(6)
                 .transform.missing.replace_na())

.with_column(Column("sb_final_best_ln_12", from_loa = "country_month", from_column = "ged_sb_best_sum_nokgi")
                 .transform.ops.ln()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(12)
                 .transform.missing.replace_na()) 
                    
.with_column(Column("sb_final_best_ln_24", from_loa = "country_month", from_column = "ged_sb_best_sum_nokgi")
                 .transform.ops.ln()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(24)
                 .transform.missing.replace_na())  
                    
# ACLED DATA                    
.with_column(Column('acled_sb_fat', from_loa = 'country_month', from_column = 'acled_sb_fat'))
                    
.with_column(Column('acled_sb_fat_ln', from_loa = 'country_month', from_column = 'acled_sb_fat')
                 .transform.ops.ln()
                 .transform.missing.fill())
                    
.with_column(Column('acled_ns_fat_ln', from_loa = 'country_month', from_column = 'acled_ns_fat')
                 .transform.ops.ln()
                 .transform.missing.fill())
                    
.with_column(Column('acled_os_fat_ln', from_loa = 'country_month', from_column = 'acled_os_fat')
                 .transform.ops.ln()
                 .transform.missing.fill())
                    
.with_column(Column('acled_sb_fat_ln_1', from_loa = 'country_month', from_column = 'acled_sb_fat')
                 .transform.ops.ln()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(1)
                 .transform.missing.replace_na()) 
                    
.with_column(Column('acled_sb_fat_ln_2', from_loa = 'country_month', from_column = 'acled_sb_fat')
                 .transform.ops.ln()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(2)
                 .transform.missing.replace_na()) 
                    
.with_column(Column('acled_sb_fat_ln_3', from_loa = 'country_month', from_column = 'acled_sb_fat')
                 .transform.ops.ln()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(3)
                 .transform.missing.replace_na()) 

.with_column(Column('acled_sb_fat_ln_12', from_loa = 'country_month', from_column = 'acled_sb_fat')
                 .transform.ops.ln()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(12)
                 .transform.missing.replace_na()) 

.with_column(Column('acled_sb_fat_ln_24', from_loa = 'country_month', from_column = 'acled_sb_fat')
                 .transform.ops.ln()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(24)
                 .transform.missing.replace_na()) 
                                       
.with_column(Column('acled_sb_count', from_loa = 'country_month', from_column = 'acled_sb_count'))
.with_column(Column('acled_ns_fat', from_loa = 'country_month', from_column = 'acled_ns_fat'))
.with_column(Column('acled_ns_count', from_loa = 'country_month', from_column = 'acled_ns_count'))
.with_column(Column('acled_os_fat', from_loa = 'country_month', from_column = 'acled_os_fat'))
.with_column(Column('acled_os_count', from_loa = 'country_month', from_column = 'acled_os_count'))
.with_column(Column('acled_pr_count', from_loa = 'country_month', from_column = 'acled_pr_count'))

.with_column(Column("ln_acled_sb_reb", from_loa="country_month", from_column="acled_bat_reb_fat")
                                    .transform.ops.ln()
                                    .transform.missing.fill()
                                    )

.with_column(Column("ln_acled_sb_gov", from_loa="country_month", from_column="acled_bat_gov_fat")
                                    .transform.ops.ln()
                                    .transform.missing.fill()
                                    )

# TOPICS DATA                                    
.with_column(Column('topic_conflict_1', from_loa = 'country_month', from_column = 'topic_ste_theta6')
                 .transform.missing.fill()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(1)
                 .transform.missing.fill())

.with_column(Column('topic_judiciary_1', from_loa = 'country_month', from_column = 'topic_ste_theta5')
                 .transform.missing.fill()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(1)
                 .transform.missing.fill())

.with_column(Column('topic_diplomacy_1', from_loa = 'country_month', from_column = 'topic_ste_theta2')
                 .transform.missing.fill()
                 .transform.missing.replace_na()
                 .transform.temporal.tlag(1)
                 .transform.missing.fill())
                    
# VDEM Data
.with_column(Column("vdem_v2x_delibdem", from_loa="country_year", from_column="vdem_v2x_delibdem")
                 .transform.missing.fill()
                 .transform.temporal.tlag(12)
                 .transform.missing.fill()
                 .transform.missing.replace_na())

.with_column(Column("vdem_v2x_clphy", from_loa="country_year", from_column="vdem_v2x_clphy")
                 .transform.missing.fill()
                 .transform.temporal.tlag(12)
                 .transform.missing.fill()
                 .transform.missing.replace_na())

.with_column(Column("vdem_v2x_rule", from_loa="country_year", from_column="vdem_v2x_rule")
                 .transform.missing.fill()
                 .transform.temporal.tlag(12)
                 .transform.missing.fill()
                 .transform.missing.replace_na())
                    
.with_column(Column("vdem_v2x_freexp", from_loa="country_year", from_column="vdem_v2x_freexp")
                 .transform.missing.fill()
                 .transform.temporal.tlag(12)
                 .transform.missing.fill()
                 .transform.missing.replace_na())


                   )
     
qs_nowcasting_cm = qs_nowcasting_cm.publish().fetch()

100%|██████████| 3.56M/3.56M [00:00<00:00, 46.3MB/s]


Queryset nowcasting read successfully 


In [9]:
ged_final_18_23 = qs_nowcasting_cm.copy()
ged_final_18_23.reset_index(inplace = True)
ged_final_18_23 = ged_final_18_23.loc[ged_final_18_23['year'] <= 2023]
ged_final_18_23 = ged_final_18_23.loc[ged_final_18_23['year'] >= 2018]
ged_final_18_23 = ged_final_18_23.rename(columns={'month_id': 'month_id', 'country_id': 'c_id'})
#nowcast = nowcast.drop(['country_name', 'iso_ab','gleditsch_ward'], axis=1)
ged_final_18_23 = ged_final_18_23.set_index(['month_id', 'c_id'])

In [10]:
# load combined_df.to_csv("candidate_combined_cm_filtered.csv") and index by , index= month_id and c_id
candidates = pd.read_csv("candidate_combined_cm_filtered.csv")

# index df by month_id and c_id
candidates = candidates.set_index(['month_id', 'c_id'])

# Rename cadidate data columns

In [11]:
# rename 'ged_sb_best_sum_nokgi' in candidates to 'candidate_sb_best_sum_nokgi'
candidates = candidates.rename(columns={'ged_sb_best_sum_nokgi': 'candidate_sb_best_sum_nokgi',
                                        'ged_ns_best_sum_nokgi': 'candidate_ns_best_sum_nokgi',
                                        'ged_os_best_sum_nokgi': 'candidate_os_best_sum_nokgi',
                                        'ged_sb_best_count_nokgi': 'candidate_sb_best_count_nokgi',
                                        'ged_ns_best_count_nokgi': 'candidate_ns_best_count_nokgi',
                                        'ged_os_best_count_nokgi': 'candidate_os_best_count_nokgi',
                                        'ged_sb_high_sum_nokgi': 'candidate_sb_high_sum_nokgi',
                                        'ged_ns_high_sum_nokgi': 'candidate_ns_high_sum_nokgi',
                                        'ged_os_high_sum_nokgi': 'candidate_os_high_sum_nokgi',
                                        'ged_sb_high_count_nokgi': 'candidate_sb_high_count_nokgi',
                                        'ged_ns_high_count_nokgi': 'candidate_ns_high_count_nokgi',
                                        'ged_os_high_count_nokgi': 'candidate_os_high_count_nokgi'})

# Join the Candidates into GED final DF

In [26]:
# Merge the dataframes on their index
master = ged_final_18_23.merge(candidates, left_index=True, right_index=True, how="left")


In [27]:
master

country_name iso_ab  gleditsch_ward  year  month  \
month_id c_id                                                            
457      1                  Guyana    GUY             110  2018      1   
         2                Suriname    SUR             115  2018      1   
         3     Trinidad and Tobago    TTO              52  2018      1   
         4               Venezuela    VEN             101  2018      1   
         5                   Samoa    WSM             990  2018      1   
...                            ...    ...             ...   ...    ...   
528      242              Tanzania    TZA             510  2023     12   
         243               Morocco    MAR             600  2023     12   
         244            Mauritania    MRT             435  2023     12   
         245                 Sudan    SDN             625  2023     12   
         246           South Sudan    SSD             626  2023     12   

               sb_final_best  ns_final_best  os_final_best  \
month_id c_id                                                
457      1                 0              0              0   
         2                 0              0              0   
         3                 0              0              0   
         4                 0              0              0   
         5                 0              0              0   
...                      ...            ...            ...   
528      242               0              0              0   
         243               4              0              0   
         244               0              0              0   
         245             364             12              8   
         246               0              0              0   

               sb_ged_count_events  ns_ged_count_events  ...  \
month_id c_id                                            ...   
457      1                       0                    0  ...   
         2                       0                    0  ...   
         3                       0                    0  ...   
         4                       0                    0  ...   
         5                       0                    0  ...   
...                            ...                  ...  ...   
528      242                     0                    0  ...   
         243                     1                    0  ...   
         244                     0                    0  ...   
         245                    12                    2  ...   
         246                     0                    0  ...   

               candidate_sb_best_count_nokgi  candidate_ns_best_count_nokgi  \
month_id c_id                                                                 
457      1                               NaN                            NaN   
         2                               NaN                            NaN   
         3                               NaN                            NaN   
         4                               NaN                            NaN   
         5                               NaN                            NaN   
...                                      ...                            ...   
528      242                             NaN                            NaN   
         243                             NaN                            NaN   
         244                             NaN                            NaN   
         245                            10.0                            3.0   
         246                             NaN                            NaN   

               candidate_os_best_count_nokgi  candidate_sb_high_sum_nokgi  \
month_id c_id                                                               
457      1                               NaN                          NaN   
         2                               NaN                          NaN   
         3                               NaN                          NaN   
      

### Accounting for the missing values from 2018-2020

2018 UCDP Candidate only covered African Countries 
2019 UCDP Candidate only covered African Countries 
2020 Jan-March: Africa; April-July Africa-midddle east; august-september Global
2021-onward: Global


In [184]:
# Define country lists
african_iso3_codes = {
    "AGO", "BDI", "BEN", "BFA", "BWA", "CAF", "CIV", "CMR", "COG", "COM", "CPV",
    "ERI", "ETH", "GAB", "GHA", "GIN", "GMB", "GNB", "GNQ", "KEN", "LBR", "LSO",
    "MDG", "MLI", "MOZ", "MRT", "MUS", "MWI", "NAM", "NER", "NGA", "RWA", "SDN",
    "SEN", "SLE", "SOM", "SSD", "STP", "SWZ", "SYC", "TCD", "TGO", "TZA", "UGA",
    "ZAF", "ZAR", "ZMB", "ZWE"
}

middle_east_iso3_codes = {
    "AFG", "ARE", "BHR", "DJI", "DZA", "EGY", "IRN", "IRQ", "ISR", "JOR", "KWT", "LBN",
    "LBY", "MAR", "MLT", "OMN", "QAT", "SAU", "SYR", "TUN", "TUR", "YEM"
}

new_countries_iso3 = {
    "ALB", "AND", "ARM", "AUT", "AZE", "BEL", "BGR", "BIH", "BLR", "CHE", "CYP", "CZE", "DEU",
    "DNK", "ESP", "EST", "FIN", "FRA", "FRO", "GBR", "GEO", "GRC", "GRL", "HRV", "HUN", "IRL",
    "ISL", "ITA", "KAZ", "KGZ", "KSV", "LIE", "LTU", "LUX", "LVA", "MCO", "MDA", "MKD", "MNT",
    "NLD", "NOR", "POL", "PRT", "ROM", "RUS", "SER", "SMR", "SVK", "SVN", "SWE", "TJK", "TKM",
    "UKR", "UZB", "BGD", "BTN", "IND", "LKA", "MDV", "NPL", "PAK", "ASM", "AUS",
    "BRN", "CHN", "FJI", "FSM", "GUM", "HKG", "IDN", "JPN", "KHM", "KIR", "KOR", "LAO", "MAC",
    "MHL", "MMR", "MNG", "MNP", "MYS", "NCL", "NZL", "PHL", "PLW", "PNG", "PRK", "PYF", "SGP",
    "SLB", "THA", "TMP", "TON", "TUV", "VNM", "VUT", "WSM"
}

In [197]:
# Copy original DataFrame
master_new = master.copy()

# Apply filtering
master_new = master_new[
    # Keep all years except 2018-2019 unless in African ISO3 codes
    (~master_new["year"].isin([2018, 2019])) |

    # Keep only African countries for 2018-2019
    (master_new["iso_ab"].isin(african_iso3_codes)) |

    # Keep African countries for Jan-Mar 2020
    ((master_new["year"] == 2020) & (master_new["month"].isin([1, 2, 3])) & master_new["iso_ab"].isin(african_iso3_codes)) |

    # Keep African and Middle Eastern countries from April to July 2020
    ((master_new["year"] == 2020) & (master_new["month"].between(4, 7)) & 
     (master_new["iso_ab"].isin(african_iso3_codes) | master_new["iso_ab"].isin(middle_east_iso3_codes))) |

    # Keep African, Middle Eastern, and new countries from August 2020 onwards
    ((master_new["year"] > 2020) |  # Keep all data from 2021 onwards
     ((master_new["year"] == 2020) & (master_new["month"] >= 8) & 
      (master_new["iso_ab"].isin(african_iso3_codes) | 
       master_new["iso_ab"].isin(middle_east_iso3_codes) | 
       master_new["iso_ab"].isin(new_countries_iso3))))
]

# Explicitly Remove Middle Eastern Countries Before April 2020
master_new = master_new[
    ~((master_new["year"] == 2020) & (master_new["month"] < 4) & master_new["iso_ab"].isin(middle_east_iso3_codes))
]

# Explicitly Remove New Countries Before August 2020
master_new = master_new[
    ~((master_new["year"] == 2020) & (master_new["month"] < 8) & master_new["iso_ab"].isin(new_countries_iso3))
]

# **Remove duplicates for April 2021 while keeping only the first occurrence of each 'iso_ab'**
mask_april_2021 = (master_new["year"] == 2021) & (master_new["month"] == 4)

# Filter only April 2021 data
april_2021_data = master_new[mask_april_2021]

# Keep the first occurrence of each 'iso_ab' and remove the rest
april_2021_unique = april_2021_data.drop_duplicates(subset=["iso_ab"], keep="first")

# Remove old April 2021 data and replace with the cleaned version
master_new = master_new[~mask_april_2021].append(april_2021_unique, ignore_index=False)


# Check the filtering results
print(f"Original rows: {len(master)}")
print(f"Filtered rows: {len(master_new)}")


Original rows: 13800
Filtered rows: 9677


In [63]:
# Copy original DataFrame
master_new = master.copy()

# Apply filtering
master_new = master_new[
    # Keep all years except 2018-2019 unless in African ISO3 codes
    (~master_new["year"].isin([2018, 2019])) |

    # Keep only African countries for 2018-2019
    (master_new["iso_ab"].isin(african_iso3_codes)) |

    # Keep African countries for Jan-Mar 2020
    ((master_new["year"] == 2020) & (master_new["month"].isin([1, 2, 3])) & master_new["iso_ab"].isin(african_iso3_codes)) |

    # Keep African and Middle Eastern countries from April to July 2020
    ((master_new["year"] == 2020) & (master_new["month"].between(4, 7)) & 
     (master_new["iso_ab"].isin(african_iso3_codes) | master_new["iso_ab"].isin(middle_east_iso3_codes))) |

    # Keep African, Middle Eastern, and new countries from August 2020 onwards
    ((master_new["year"] > 2020) |  # Keep all data from 2021 onwards
     ((master_new["year"] == 2020) & (master_new["month"] >= 8) & 
      (master_new["iso_ab"].isin(african_iso3_codes) | 
       master_new["iso_ab"].isin(middle_east_iso3_codes) | 
       master_new["iso_ab"].isin(new_countries_iso3))))
]

# Explicitly Remove Middle Eastern Countries Before April 2020
master_new = master_new[
    ~((master_new["year"] == 2020) & (master_new["month"] < 4) & master_new["iso_ab"].isin(middle_east_iso3_codes))
]

# Explicitly Remove New Countries Before August 2020
master_new = master_new[
    ~((master_new["year"] == 2020) & (master_new["month"] < 8) & master_new["iso_ab"].isin(new_countries_iso3))
]

# Check the filtering results
print(f"Original rows: {len(master)}")
print(f"Filtered rows: {len(master_new)}")


Original rows: 13800
Filtered rows: 9725


# Fill all missing with 0?

In [199]:
# drop rows with 'country_name' == 'Kosovo'
master_new = master_new[master_new['country_name'] != 'Kosovo']

# replace missing values in master with '0'
master_new = master_new.fillna(0)


In [200]:
# Feature engineering
master_new['candidate_sb_best_sum_nokgi_ln1'] = np.log1p(master_new['candidate_sb_best_sum_nokgi'])
master_new['candidate_ns_best_sum_nokgi_ln1'] = np.log1p(master_new['candidate_ns_best_sum_nokgi'])
master_new['candidate_os_best_sum_nokgi_ln1'] = np.log1p(master_new['candidate_os_best_sum_nokgi'])

master_new['candidate_sb_high_sum_nokgi_ln1'] = np.log1p(master_new['candidate_sb_high_sum_nokgi'])
master_new['candidate_ns_high_sum_nokgi_ln1'] = np.log1p(master_new['candidate_ns_high_sum_nokgi'])
master_new['candidate_os_high_sum_nokgi_ln1'] = np.log1p(master_new['candidate_os_high_sum_nokgi'])

one_hot = pd.get_dummies(master_new['iso_ab'])

# drop the 'source_version' column
master_new = master_new.drop(['source_version'], axis=1)

# Join the one-hot encoded columns back with the original DataFrame
master_new = master_new.join(one_hot)

master_new.columns = master_new.columns.astype(str)

In [201]:
master_new.to_csv('nowcasting_master_data.csv', index=True)

# Adding the Candidate Reivewed Data to the mix

There are 3 types of data
1. Candidate -- the most recent data
2. Candidate Reviewed -- data that is inputted into the candidate dataset at the end of the year-- corrects for mis-counts earlier in the year
3. Final -- GED final data that is the "ground truth"

The current master contains the "candidate" and the "final" yet I do not yet have the "candidate reviewed" I add that now

In [202]:
# load combined_df.to_csv("candidate_combined_cm_filtered.csv") and index by , index= month_id and c_id
candidate_reviewed = pd.read_csv("candidate_finals_combined_cm.csv")

# index df by month_id and c_id
candidate_reviewed = candidate_reviewed.set_index(['month_id', 'c_id'])

In [203]:
# rename 'ged_sb_best_sum_nokgi' in candidates to 'candidate_reviewed_sb_best_sum_nokgi'
candidate_reviewed = candidate_reviewed.rename(columns={'ged_sb_best_sum_nokgi': 'candidate_reviewed_sb_best_sum_nokgi',
                                                        'ged_ns_best_sum_nokgi': 'candidate_reviewed_ns_best_sum_nokgi',
                                                        'ged_os_best_sum_nokgi': 'candidate_reviewed_os_best_sum_nokgi',
                                                        'ged_sb_best_count_nokgi': 'candidate_reviewed_sb_best_count_nokgi',
                                                        'ged_ns_best_count_nokgi': 'candidate_reviewed_ns_best_count_nokgi',
                                                        'ged_os_best_count_nokgi': 'candidate_reviewed_os_best_count_nokgi',
                                                        'ged_sb_high_sum_nokgi': 'candidate_reviewed_sb_high_sum_nokgi',
                                                        'ged_ns_high_sum_nokgi': 'candidate_reviewed_ns_high_sum_nokgi',
                                                        'ged_os_high_sum_nokgi': 'candidate_reviewed_os_high_sum_nokgi',
                                                        'ged_sb_high_count_nokgi': 'candidate_reviewed_sb_high_count_nokgi',
                                                        'ged_ns_high_count_nokgi': 'candidate_reviewed_ns_high_count_nokgi',
                                                        'ged_os_high_count_nokgi': 'candidate_reviewed_os_high_count_nokgi'})

# Join these data to the master

In [204]:
# Merge the dataframes on their index
master_final = master_new.merge(candidate_reviewed, left_index=True, right_index=True, how="left")


In [205]:
# replace missing values in master with '0'
master_final = master_final.fillna(0)

master_final['candidate_reviewed_sb_best_sum_nokgi_ln1'] = np.log1p(master_final['candidate_reviewed_sb_best_sum_nokgi'])
master_final['candidate_reviewed_ns_best_sum_nokgi_ln1'] = np.log1p(master_final['candidate_reviewed_ns_best_sum_nokgi'])
master_final['candidate_reviewed_os_best_sum_nokgi_ln1'] = np.log1p(master_final['candidate_reviewed_os_best_sum_nokgi'])

master_final['candidate_reviewed_sb_high_sum_nokgi_ln1'] = np.log1p(master_final['candidate_reviewed_sb_high_sum_nokgi'])
master_final['candidate_reviewed_ns_high_sum_nokgi_ln1'] = np.log1p(master_final['candidate_reviewed_ns_high_sum_nokgi'])
master_final['candidate_reviewed_os_high_sum_nokgi_ln1'] = np.log1p(master_final['candidate_reviewed_os_high_sum_nokgi'])



In [206]:
master_final_1 = master_final.copy()

In [207]:
# Apply lagging within each country group
master_final_1['candidate_sb_best_sum_nokgi_ln1_lag1'] = master_final_1.groupby('country_name')['candidate_sb_best_sum_nokgi_ln1'].shift(1)
master_final_1['candidate_sb_best_sum_nokgi_ln1_lag2'] = master_final_1.groupby('country_name')['candidate_sb_best_sum_nokgi_ln1'].shift(2)
master_final_1['candidate_sb_best_sum_nokgi_ln1_lag3'] = master_final_1.groupby('country_name')['candidate_sb_best_sum_nokgi_ln1'].shift(3)


In [208]:
# drop rows with nan values
master_final_1 = master_final_1.dropna()


# SAVING THE MAASTERRRR 

In [209]:
master_final_1.to_csv('nowcasting_master_final_data_1.csv', index=True)